In [5]:
!pip install path

In [7]:
#Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
from path import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans


In [8]:
file = Path('crypto_data.csv')

In [9]:
#Read in data, create df
df = pd.read_csv(file, index_col=0)
df.head()

CoinName Algorithm  IsTrading ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt       True   PoW/PoS     4.199995e+01              42
365    365Coin       X11       True   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt       True   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       True       PoW              NaN          611000
808        808   SHA-256       True   PoW/PoS     0.000000e+00               0

In [11]:
#List algorithms in use
algo_list = df['Algorithm'].values
print(algo_list)

['Scrypt' 'X11' 'Scrypt' ... 'Scrypt' 'SHA-256' 'PoS']


In [12]:
#Check stocks that are trading
df.IsTrading.value_counts()

True     1144
False     108
Name: IsTrading, dtype: int64

In [17]:
#Replace with only actively trading stocks
df = df[df['IsTrading']==True]
df['IsTrading'].value_counts()

True    1144
Name: IsTrading, dtype: int64

In [18]:
#Delete IsTrading series
df = df.drop('IsTrading', axis=1)
df.head()

CoinName Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42     42 Coin    Scrypt   PoW/PoS     4.199995e+01              42
365    365Coin       X11   PoW/PoS              NaN      2300000000
404    404Coin    Scrypt   PoW/PoS     1.055185e+09       532000000
611  SixEleven   SHA-256       PoW              NaN          611000
808        808   SHA-256   PoW/PoS     0.000000e+00               0

In [21]:
#Drop all rows with null values
df=df.dropna()
df.shape

(685, 5)

In [23]:
#Sort by TotalCoinsMined
df.TotalCoinsMined.sort_values()

FIII   -5.917978e+09
LBTC    0.000000e+00
RIPO    0.000000e+00
BASH    0.000000e+00
CSH     0.000000e+00
            ...     
QWC     9.955311e+10
NYC     1.430067e+11
GCN     1.630551e+11
BCN     1.840668e+11
BTT     9.899887e+11
Name: TotalCoinsMined, Length: 685, dtype: float64

In [24]:
#Drop non-positive values
df = df.loc[df['TotalCoinsMined']>0]
len(df)

532

In [25]:
#Drop CoinName column
df = df.drop('CoinName', axis=1)
df.head()

Algorithm ProofType  TotalCoinsMined TotalCoinSupply
42      Scrypt   PoW/PoS     4.199995e+01              42
404     Scrypt   PoW/PoS     1.055185e+09       532000000
1337       X13   PoW/PoS     2.927942e+10    314159265359
BTC    SHA-256       PoW     1.792718e+07        21000000
ETH     Ethash       PoW     1.076842e+08               0

In [26]:
#Create dummies for categorical variables
X = pd.get_dummies(data=df, columns=['Algorithm', 'ProofType'])
print(X.shape)
X.head()

(532, 98)


TotalCoinsMined TotalCoinSupply  Algorithm_1GB AES Pattern Search  \
42       4.199995e+01              42                                 0   
404      1.055185e+09       532000000                                 0   
1337     2.927942e+10    314159265359                                 0   
BTC      1.792718e+07        21000000                                 0   
ETH      1.076842e+08               0                                 0   

      Algorithm_536  Algorithm_Argon2d  Algorithm_BLAKE256  Algorithm_Blake  \
42                0                  0                   0                0   
404               0                  0                   0                0   
1337              0                  0                   0                0   
BTC               0                  0                   0                0   
ETH               0                  0                   0                0   

      Algorithm_Blake2S  Algorithm_Blake2b  Algorithm_C11  ...  \
42                    0                  0              0  ...   
404                   0                  0              0  ...   
1337                  0                  0              0  ...   
BTC                   0                  0              0  ...   
ETH                   0                  0              0  ...   

      ProofType_PoW/PoS  ProofType_PoW/PoS   ProofType_PoW/PoW  \
42                    1                   0                  0   
404                   1                   0                  0   
1337                  1                   0                  0   
BTC                   0                   0                  0   
ETH                   0                   0                  0   

      ProofType_PoW/nPoS  ProofType_Pos  ProofType_Proof of Authority  \
42                     0              0                             0   
404                    0              0                             0   
1337                   0              0                             0   
BTC                    0              0                             0   
ETH                    0              0                             0   

      ProofType_Proof of Trust  ProofType_TPoS  \
42                           0               0   
404                          0               0   
1337                         0               0   
BTC                          0               0   
ETH                          0               0   

      ProofType_Zero-Knowledge Proof  ProofType_dPoW/PoW  
42                                 0                   0  
404                                0                   0  
1337                               0                   0  
BTC                                0                   0  
ETH                                0                   0  

[5 rows x 98 columns]

In [29]:
#Apply Standard Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[0]

array([-0.11710817, -0.1528703 , -0.0433963 , -0.0433963 , -0.0433963 ,
       -0.06142951, -0.07530656, -0.0433963 , -0.06142951, -0.06142951,
       -0.0433963 , -0.0433963 , -0.19245009, -0.06142951, -0.09740465,
       -0.0433963 , -0.11547005, -0.07530656, -0.0433963 , -0.0433963 ,
       -0.15191091, -0.0433963 , -0.13118084, -0.0433963 , -0.0433963 ,
       -0.08703883, -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
       -0.06142951, -0.0433963 , -0.08703883, -0.08703883, -0.08703883,
       -0.0433963 , -0.13118084, -0.13840913, -0.13840913, -0.0433963 ,
       -0.06142951, -0.0433963 , -0.07530656, -0.18168574, -0.0433963 ,
       -0.0433963 , -0.0433963 , -0.07530656, -0.15826614, -0.31491833,
       -0.0433963 , -0.08703883, -0.07530656, -0.06142951,  1.38675049,
       -0.0433963 , -0.0433963 , -0.06142951, -0.0433963 , -0.0433963 ,
       -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 , -0.0433963 ,
       -0.0433963 , -0.39879994, -0.0433963 , -0.18168574, -0.04

In [30]:
#Idendify nuumber of rows and columns in scaled data
X_scaled.shape

(532, 98)

In [32]:
#Use PCA to conduct dimmensionality reduction
pca = PCA(n_components=0.99)
X_pca = pca.fit_transform(X_scaled)
X_pca.shape

(532, 86)

In [33]:
#Use PCA to conduct dimmensionality reduction, reduce components further
pca = PCA(n_components=0.90)
X_pca = pca.fit_transform(X_scaled)
X_pca.shape

(532, 74)

In [ ]:
#NOTE: As the explained variance preserved (n_components) decreased, the 
#      number of components also decreased.